In [1]:
import pandas as pd

## Participant

# Questions to answer
- How many participants in total (Unique)
-- Friday
-- Regular

In [8]:
# Participants
regular_file = "/home/yusuf/keminggris/data/regular_sesh/Keminggris - Session Registration Form (Responses) - Form Responses 1.csv"
friday_file = "/home/yusuf/keminggris/data/friday_sesh/Keminggris X Perpus Cafe August 2025 - Special Session - Participant Responses.csv"

regular_df = pd.read_csv(regular_file)
friday_df = pd.read_csv(friday_file)

In [2]:
import re
import unicodedata

def _ascii(s):
    return unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')

def _standardize_name(s):
    s = _ascii(str(s))
    s = s.lower()
    s = re.sub(r'[\r\n]+', ' ', s)
    s = re.sub(r'[^a-z0-9]+', '_', s)
    s = re.sub(r'_+', '_', s).strip('_')
    return s[:48]

def _map_short(norm):
    if 'topic' in norm:
        return 'topic_suggestion'
    if 'your_name' in norm:
        return 'name'
    if 'your_email' in norm:
        return 'email'
    if 'upcoming_sessions' in norm or 'sessions_you_ll_join' in norm or 'sessions' in norm:
        return 'sessions_joining'
    if 'english_level' in norm:
        return 'english_level'
    if 'made_you_want_to_join' in norm or 'keminggris_session' in norm or 'want_to_join' in norm:
        return 'motivation'
    if 'hear_about_us' in norm or 'how_you_found_us' in norm or 'first_hear' in norm or 'found_us' in norm:
        return 'discovery_source'
    if 'insta' in norm or 'instagram' in norm:
        return 'instagram'
    if 'any_thoughts_suggestions_or_even_criticism_for_t' in norm:
        return 'moderator_suggestions'
    if 'any_thoughts_suggestions' in norm:
        return 'suggestions'
    if 'whole_session' in norm:
        return 'overal_session_feedback'
    if 'helped_you_become_more_confiden' in norm:
        return 'confidence_feedback'
    if 'comfortable' in norm:
        return 'comfortable_feedback'
    if 'are_you_interested_in_joining_our_next_session' in norm:
        return 'interested_in_next_session'
    if 'shoutout' in norm:
        return 'shoutout'
    if 'name_of_your_momod' in norm:
        return 'moderator_name'

    if norm == 'timestamp':
        return 'timestamp'
    return norm

def standardize_columns(df):
    cols = []
    for c in df.columns:
        norm = _standardize_name(c)
        short = _map_short(norm)
        cols.append(short)
    df = df.copy()
    df.columns = cols
    return df




In [51]:
feedback_df.columns

Index(['timestamp', 'overal_session_feedback', 'confidence_feedback',
       'comfortable_feedback', 'interested_in_next_session',
       'any_thoughts_suggestions_or_even_criticism_were_',
       'what_s_your_linkedin_url', 'column_1'],
      dtype='object')

In [ ]:
regular_df = standardize_columns(regular_df)
friday_df = standardize_columns(friday_df)

cols_to_drop_regular = ['what_s_your_linkedin_url']
cols_to_drop_friday = ['as_part_of_this_special_session_collab_we_kindly', 'have_you_ever_joined_a_friday_session_before_thi', 'column_10']

regular_df = regular_df.drop(cols_to_drop_regular, axis=1)
friday_df = friday_df.drop(cols_to_drop_friday, axis=1)

regular_df['session_type'] = 'regular'
friday_df['session_type'] = 'friday'

participants_df = pd.concat([regular_df, friday_df])


In [16]:
participants_df['discovery_source'] = participants_df['discovery_source'].replace({
    'Tiktok': 'TikTok',
    'Yudid': 'Already know the team',
    'Ekky': 'Already know the team',
    'mrc whatsapp group': 'Malang Remote Club',
    'MRC': 'Malang Remote Club',
    'wfc community - Malang Remote Club': 'Malang Remote Club',
    'Yudhis': 'Already know the team',


})

In [18]:
participants_df.to_csv('participants.csv')

## Session feedback

In [3]:
feedback_file = "/home/yusuf/keminggris/data/regular_sesh/Keminggris - Session Feedback - Form Responses 1.csv"
feedback_file_2 = "/home/yusuf/keminggris/data/regular_sesh/Keminggris - Session Feedback - Form Responses 2.csv"

feedback_df = pd.read_csv(feedback_file)
feedback_df_2 = pd.read_csv(feedback_file_2)




In [4]:
feedback_df.shape
feedback_df_2.shape

(73, 9)

In [5]:
feedback_df = standardize_columns(feedback_df)
feedback_df = feedback_df.drop(columns=['column_1', 'what_s_your_linkedin_url'])
feedback_df_2 = standardize_columns(feedback_df_2)

feedback_df = pd.concat([feedback_df, feedback_df_2])

In [14]:
feedback_df.query("confidence_feedback == 2")

,timestamp,overal_session_feedback,confidence_feedback,comfortable_feedback,interested_in_next_session,suggestions,shoutout,moderator_name,moderator_suggestions
63,8/29/2025 21:08:59,5.0,2.0,4.0,YES!!!,I want to challenge my thoughts pattern. Aku p...,NaN,NaN,NaN


In [16]:
feedback_df.columns

Index(['timestamp', 'overal_session_feedback', 'confidence_feedback',
       'comfortable_feedback', 'interested_in_next_session', 'suggestions',
       'shoutout', 'moderator_name', 'moderator_suggestions'],
      dtype='object')

In [19]:
feedback_df[feedback_df['interested_in_next_session'] == 'Not Really~']

,timestamp,overal_session_feedback,confidence_feedback,comfortable_feedback,interested_in_next_session,suggestions,shoutout,moderator_name,moderator_suggestions
19,9/26/2025 20:31:00,3.0,5.0,3.0,Not Really~,Saya menyarankan menyelesaikan pekerjaan masin...,"Mas hadyan, mbak mia, mbak azmi",Azmi,NaN


## Model Sentiment Analysis

In [7]:
import pandas as pd
import numpy as np
import re


In [ ]:
import re
import pandas as pd

# -----------------------------
# Load CSV (needs 'suggestions')
# -----------------------------
df = pd.read_csv("feedback.csv")

def _norm_yes(v: object) -> str:
    s = str(v).strip().lower()
    if s in {"yes", "yes!!!", "y", "ya", "iya", "ok", "sure", "of course"}: return "Yes"
    if s in {"no", "n", "tidak", "nggak", "ga", "nope", "not really~", "not really"}: return "No"
    return "Unknown"

def _norm_cols(df):
    # Flexible renames (case-insensitive + typos)
    rename = {
        "timestamp": "timestamp", "Timestamp": "timestamp",
        "overal_session_feedback": "overall", "overall_session_feedback": "overall",
        "confidence_feedback": "confidence",
        "comfortable_feedback": "comfortable",
        "interested_in_next_session": "interested_next",
        "suggestions": "suggestions",
        "shoutout": "shoutout",
        "moderator_name": "moderator_name",
        "moderator_suggestions": "moderator_suggestions",
    }
    cols = {c: rename.get(c, rename.get(c.lower(), c)) for c in df.columns}
    return df.rename(columns=cols)

def _clean_text(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    return s if s else np.nan

df = _norm_cols(df)

keep = [
    "timestamp", "overall", "confidence", "comfortable",
    "interested_next", "suggestions", "shoutout",
    "moderator_name", "moderator_suggestions"
]
for k in keep:
    if k not in df.columns: df[k] = np.nan
df = df[keep]

# Parse time
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
# Adjust specific dates by shifting timestamp back one day (preserves time)
mask_0921 = df["timestamp"].notna() & (df["timestamp"].dt.date == pd.to_datetime("2025-09-21").date())
mask_1019 = df["timestamp"].notna() & (df["timestamp"].dt.date == pd.to_datetime("2025-10-19").date())
df.loc[mask_0921, "timestamp"] = df.loc[mask_0921, "timestamp"] - pd.Timedelta(days=1)
df.loc[mask_1019, "timestamp"] = df.loc[mask_1019, "timestamp"] - pd.Timedelta(days=1)
df["date"] = df["timestamp"].dt.date
df["month"] = df["timestamp"].dt.to_period("M").astype(str)
# Session label only for Friday/Saturday
mask_session = df["timestamp"].notna() & df["timestamp"].dt.day_name().isin(["Friday", "Saturday"])
df["session"] = np.nan
df.loc[mask_session, "session"] = df.loc[mask_session, "timestamp"].dt.strftime("%A, %d %B %Y")

df["session_day"] = np.nan
dayname = df["timestamp"].dt.day_name()
df.loc[dayname == "Friday", "session_day"] = "Friday"
df.loc[dayname == "Saturday", "session_day"] = "Regular"

# Ratings to numeric, bound to [1,5]
for c in ["overall", "confidence", "comfortable"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
    df.loc[(df[c] < 1) | (df[c] > 5), c] = np.nan

# Normalize booleans / interest
df["interested_next_norm"] = df["interested_next"].apply(_norm_yes)

# Clean texts
for c in ["suggestions", "shoutout", "moderator_name", "moderator_suggestions"]:
    df[c] = df[c].apply(_clean_text)


# Strip moderator names & harmonize small variants
if df["moderator_name"].notna().any():
    df["moderator_name"] = df["moderator_name"].str.replace(r"\s+", " ", regex=True).str.strip()
    # Optional: unify emojis/suffixes like "Yudis 🙌🏻" -> "Yudis"
    df["moderator_name"] = df["moderator_name"].str.replace(r"[^A-Za-z\s&]+", "", regex=True).str.strip()
    # Merge obvious near-duplicates
    df["moderator_name"] = df["moderator_name"].replace({
        "Ucup": "Ucup",
        "Ucup  ": "Ucup",
        "Ucup  ": "Ucup",
        "Yudis": "Yudis",
        "Yudis  ": "Yudis",
        "Lilla": "Lilla", "Lila": "Lilla",
        "Tasya  Tifa": "Tasya & Tifa", "Tasya   Tifa": "Tasya & Tifa",
        "Tasya  Tifa": "Tasya & Tifa",
        "Tasya  Tifa ": "Tasya & Tifa",
        "Vincent": "Vincent", "Niko": "Niko", "Mr Yusuf": "Yusuf", "Sheehan": "Sheehan",
        "Aulia": "Aulia", "Azmi": "Azmi", "Ekky": "Ekky", "Tifa": "Tifa",
    })

    

    # Drop rows with no rating info at all (trailing junk lines)
    

# If your column name casing varies, normalize:
col_map = {c.lower(): c for c in df.columns}
if "suggestions" not in col_map:
    raise ValueError("CSV must contain a 'suggestions' column.")
SUG = col_map["suggestions"]
SUG_MOD = col_map["moderator_suggestions"]

# Optional: basic cleaning helper
def clean_text(x: str) -> str:
    if not isinstance(x, str): return ""
    # collapse spaces, strip
    return re.sub(r"\s+", " ", x).strip()

df[SUG] = df[SUG].fillna("").map(clean_text)
df[SUG_MOD] = df[SUG_MOD].fillna("").map(clean_text)

# -----------------------------
# Sentiment rules (EN + ID)
# Precedence: Negative > Constructive > Positive > Neutral
# -----------------------------
NEG_PAT = re.compile(
    r"(?i)\b("
    r"boring|bad|gloomy|confusing|uncomfortable|not comfortable|"
    r"too (noisy|crowded|late|many openings?)|too much opening|:\(\|"
    r"ramai|berisik|terlalu malam|membosankan|buruk|tidak nyaman|kurang nyaman|pelan|"
    r"nggak nyaman|gloomy"
    r")\b"
)

SUG_PAT = re.compile(
    r"(?i)\b("
    # generic asks/hedges
    r"maybe|please|could|should|would|wish|hope|mungkin|"
    # time/duration/frequency
    r"longer|more time|extend(ed)? (time|duration)|"
    r"(two|three)\s+times a week|one week\s*2\s*times|"
    r"seminggu\s*(2|dua|3|tiga)|lebih sering|sering2|"
    # add/change/things
    r"add|increase|earlier|topic|topics|spicier|fixed topic|games?|game session|"
    r"switch|rotate|rotation|bilingual|translate|translation|"
    r"quota|kuota|seat|waiting list|wait.?list|war\b|"
    r"whatsapp|wa|group chat|native|mentor"
    r"mungkin|harap|sebaiknya|lebih|tambah|panjang(i|in)? waktu|kasih"
    r")\b"
)

PRAISE_PAT = re.compile(
    r"(?i)\b("
    r"love|great|awesome|amazing|fantastic|fun|enjoy|nice|"
    r"well[- ]?organized|helpful|mantap|keren|bagus|seru|as(y|i)k|asik|"
    r"totally fun|so fun|incredible|perfect|all good|all great|"
    r"5/5|pretty fun|it'?s fun|i'?m enjoy|enjoyed"
    r")\b"
)

def label_sentiment(text: str) -> str:
    if not text: return "Neutral"
    neg = bool(NEG_PAT.search(text.lower()))
    sug = bool(SUG_PAT.search(text.lower()))
    pos = bool(PRAISE_PAT.search(text.lower()))
    if neg:
        return "Negative"
    if sug:
        return "Constructive"
    if pos:
        return "Positive"
    return "Neutral"

# -----------------------------
# Theme rules (multi-label)
# -----------------------------
THEMES = {
    "More time / longer duration": [
        r"\blonger (time|session)s?\b", r"\bmore time\b",
        r"\bextend(ed)? (time|duration)\b", r"\bpanjang(i|in)? waktu\b", r"\btambah waktu\b"
    ],
    "More sessions / higher frequency": [
        r"\bmore (session|meet(ing)?s)\b", r"\b(two|three)\s+times a week\b",
        r"\bseminggu\s*(2|dua|3|tiga)\b", r"\b(one week|a week)\s*2\s*times\b", r"\blebih sering|sering2\b"
    ],
    "Games / activities": [
        r"\bgames?\b", r"\bmini games?\b", r"\bhangman\b", r"\brole ?play\b|\bRP\b",
        r"\bactivities?\b", r"\bgame session\b", r"\bmore fun\b"
    ],
    "Earlier or daytime schedule": [
        r"\bmorning\b|\bafternoon\b|\bdaytime\b|\bearlier\b",
        r"\bpagi\b|\bsiang\b|\bsore\b|\bmalam2\b|\bmalam\b|\b(jangan|tidak)\s+malam\b"
    ],
    "Topics / learning content": [
        r"\btopic(s)?\b|\bspicier topic\b|\bfixed topic\b|\bcontent\b|\btopik\b",
        r"\blearning|vocab|grammar|edukasi|review\b",
        r"\bbilingual|translate|translation\b|\bmore clearly\b"
    ],
    "Social / dinner / mingle": [
        r"\bdinner\b|\bsnack(s)?\b|\bcoffee\b|\bmingl(e|ing)\b|\bkumpul\b|\boutdoor\b|\bworkshop\b"
    ],
    "Group size / rotation": [
        r"\bswitch(ing)? (group|teams)\b|\brotate|rotation\b|\b(bigger|larger) group\b"
    ],
    "Capacity / quota / registration": [
        r"\bquota\b|\bkuota\b|\bseat(s)?\b|\b(waiting|wait) ?list\b|\bwar\b|\bchallenge getting a seat\b"
    ],
    "Community / WhatsApp / contacts": [
        r"\bwhats(app)?\b|\bWA\b|\bgroup chat\b|\bkeep in touch\b|\bcontact\b"
    ],
    "Noise / crowded / room comfort": [
        r"\btoo (noisy|crowded)\b|\bberisik\b|\bramai\b|\bnot comfortable\b|\btidak nyaman\b"
    ],
    "Moderator facilitation": [
        r"\btoo much opening(s)?\b|\bmore space to talk\b|\bencourage others\b|\b(dominate|dominated)\b"
    ],
    "Native speakers / mentors": [
        r"\bnative(s)?\b|\bmentor(s)?\b"
    ],
}
COMPILED_THEMES = {k: [re.compile(p, re.I) for p in v] for k, v in THEMES.items()}

def tag_themes(text: str):
    if not text: return []
    hits = [lab for lab, pats in COMPILED_THEMES.items() if any(p.search(text) for p in pats)]
    return hits

# -----------------------------
# Apply & create new columns
# -----------------------------
df["sentiment"] = df[SUG].apply(label_sentiment)
df['moderator_sentiment'] = df[SUG_MOD].apply(label_sentiment)
df["themes"] = df[SUG].apply(lambda s: "; ".join(tag_themes(s)))

has_any = df[["overall", "confidence", "comfortable"]].notna().any(axis=1)
df = df.loc[has_any & df["timestamp"].notna()].copy()

# Preview & quick counts
display(df[[SUG,  "sentiment", "themes", SUG_MOD, "moderator_sentiment"]].head(25))
print("\nSentiment counts:")
print(df["sentiment"].value_counts(dropna=False).to_string())

print("\nTop themes:")
theme_counts = df["themes"].str.get_dummies(sep="; ").sum().sort_values(ascending=False)
display(theme_counts.head(20))

# Save if needed
# df.to_csv("feedback_tagged.csv", index=False)


/tmp/ipykernel_11943/3147787530.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Saturday, 19 July 2025' 'Saturday, 19 July 2025'
 'Saturday, 19 July 2025' 'Saturday, 19 July 2025'
 'Saturday, 19 July 2025' 'Saturday, 19 July 2025'
 'Friday, 01 August 2025' 'Friday, 01 August 2025'
 'Friday, 01 August 2025' 'Friday, 01 August 2025'
 'Friday, 01 August 2025' 'Friday, 01 August 2025'
 'Friday, 01 August 2025' 'Friday, 01 August 2025'
 'Friday, 01 August 2025' 'Friday, 01 August 2025'
 'Friday, 01 August 2025' 'Saturday, 09 August 2025'
 'Saturday, 09 August 2025' 'Saturday, 09 August 2025'
 'Saturday, 09 August 2025' 'Saturday, 09 August 2025'
 'Saturday, 09 August 2025' 'Saturday, 09 August 2025'
 'Saturday, 09 August 2025' 'Saturday, 09 August 2025'
 'Saturday, 09 August 2025' 'Friday, 15 August 2025'
 'Friday, 15 August 2025' 'Friday, 15 August 2025'
 'Friday, 15 August 2025' 'Friday, 15 August 20

,suggestions,sentiment,themes,moderator_suggestions,moderator_sentiment
0,,Neutral,,,Neutral
1,We should talk about literally anything,Constructive,,,Neutral
2,,Neutral,,,Neutral
3,,Neutral,,,Neutral
4,Mungkin acaranya jangan malam2 kalau bisa hehe,Constructive,Earlier or daytime schedule,,Neutral
5,Kasih topik sebelum mulai supaya group discuss...,Constructive,Topics / learning content,,Neutral
6,,Neutral,,,Neutral
7,,Neutral,,,Neutral
8,It's some great new experience:),Positive,,,Neutral
9,No way,Negative,,,Neutral



Sentiment counts:
sentiment
Neutral         76
Constructive    52
Positive        24
Negative         5

Top themes:


Games / activities                  14
Topics / learning content           11
More time / longer duration          7
Social / dinner / mingle             6
Earlier or daytime schedule          4
Capacity / quota / registration      3
More sessions / higher frequency     3
Group size / rotation                2
Moderator facilitation               2
Community / WhatsApp / contacts      1
Noise / crowded / room comfort       1
Native speakers / mentors            1
dtype: int64

In [65]:
df['themes'].value_counts()

themes
                                                                                    106
Games / activities                                                                   13
Topics / learning content                                                            11
Social / dinner / mingle                                                              6
More time / longer duration                                                           5
Earlier or daytime schedule                                                           3
More sessions / higher frequency                                                      3
Capacity / quota / registration                                                       3
Noise / crowded / room comfort                                                        1
Games / activities; Group size / rotation                                             1
More time / longer duration; Group size / rotation                                    1
Native speakers / mentors

In [66]:
df[df['sentiment'] == 'Negative']['suggestions'].tolist()

['No way',
 'I think this is good idea, but i feel so crowded and get so many voices at one room, so i need to speak louder and that makes us not comfortable tbh, but overall so good',
 'I think we need a lot more space for conversation instead of a lot of openings. Too much opening this session.',
 'why choose the gloomy topic tho😔 we came for fun😔 BUT STILL thanks for making this safety place for yap💓']

In [71]:
df.to_csv('feedback.csv')

In [21]:
df[['suggestions', 'sentiment', 'themes']].to_csv('feedback_tagged.csv', index=False)

In [17]:
feedback_df.to_csv('feedback.csv')